<a href="https://colab.research.google.com/github/dlkt101101/CS431/blob/main/A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CS431/631 Big Data Infrastructure
### Winter 2025 - Assignment 1
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** _replace this with your name_
* **ID:** _replace this with your UW student ID number_

---
#### Overview
For this assignment, you will be using Python to analyze the [pointwise mutual information (PMI)](http://en.wikipedia.org/wiki/Pointwise_mutual_information) of tokens in the text of Shakespeare's plays.    For this assignment, you will need the same text file (`Shakespeare.txt`) that you used for Assignment 0.   You will also need the Python tokenizer module, `simple_tokenize.py`.

If two events $x$ and $y$ are independent, their PMI will be zero.   A positive PMI indicates that $x$ and $y$ are more likely to co-occur than they would be if they were independent.   Similarly, a negative PMI indicates that $x$ and $y$ are less likely to co-occur.   The PMI of events $x$ and $y$ is given by
\begin{equation*}
PMI(x,y) = \log\frac{p(x,y)}{p(x)p(y)}
\end{equation*}
where $p(x)$ and $p(y)$ are the probabilities of occurrence of events $x$ and $y$, and $p(x,y)$ is the probability of co-occurrence of $x$ and $y$.

For this assignment, the "events" that we are interested in are occurrences of tokens on lines of text in the input file.   For example, one event
might represent the occurence of the token "fire" a line of text, and another might represent the occurrence of the token "peace".   In that case, $p(fire)$ represents the probability that "fire" will occur on a line of text, and $p(fire,peace)$ represents the probability that *both* "fire" and "peace" will occur on the *same* line.   For the purposes of these PMI computations, it does not matter how many times a given token occures on a single line.   Either a line contains a particular token (at least once), or it does not.   For example, consider this line of text:

> three three three, said thrice

For this line, the following token-pair events have occurred:
- (three, said)
- (three, thrice)
- (said, three)
- (said, thrice)
- (thrice, three)
- (thrice, said)

Note that we are not interested in "reflexive" pairs, such as (thrice,thrice).

In addition to the probabilities of events, we will also be interested in the absolute *number* of occurences of particular events, e.g., the number of lines in which "fire" occurs.   We will use $n(x)$ to represent the these numbers.

Your main task for this assignment is to write Python code to analyze the PMI of tokens from Shakespeare's plays.    Based this analysis, we want to be able to answer two types of queries:

* Two-Token Queries: Given a pair of tokens, $x$ and $y$, report the number of lines on which that pair co-occurs ($n(x,y)$) as well as $PMI(x,y)$.
* One-Token Queries: Given a single token, $x$, report the number of lines on which that token occurs ($n(x)$).   In addition, report the five tokens that have the largest PMI with respect to $x$ (and their PMIs).   That is, report the five $y$'s for which $PMI(x,y)$ is largest.

To avoid reporting spurious results for the one-token queries, we are only interested in token pairs that co-occur a sufficient number of times.   Therefore, we will use a *threshold* parameter for one-token queries.   A one-token query should only report pairs of tokens that co-occur at least *threshold* times in the input.   For example, given the threshold 12, a one-token query for "fire" the should report the five tokens that have the largest PMI (with respect to "fire") among all tokens that co-occur with "fire" on at least 12 lines.   If there are fewer than five such tokens, report fewer than five.



Run the next block to download the text file (`Shakespeare.txt`) and the Python tokenizer module (`simple_tokenize.py`).

In [1]:
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/Shakespeare.txt
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/simple_tokenize.py

---
#### Question 1  (2/10 marks):

Before writing code to handle the PMI queries, start writing some code to answer some simpler questions that give an
idea of how big the PMI analysis problem will be.   The box below contains some starter code that reads in the 'Shakespeare.txt' file and tokenizes it one line at time.   (This is the same code you started with for Assignment 0.)  Extend this code to determine (a) the number of *distinct* tokens that exist in 'Shakespeare.txt', and (b) the number of
*distinct* token pairs that exist in 'Shakespeare.txt'.  For the purposes of this question, consider the token pair $x,y$ to be distinct from the pair $y,x$, i.e., count them both.   Ignore token pairs of the form $x,x$. Note that we're considering things one line at a time, so the number of unique pairs is NOT (n)(n-1) where n is the number of unique tokens! If a line is "this is fine" then there are 6 distinct pairs on that line.

As a sanity check, when rounded you should be getting ~26K distinct tokens and ~2M distinct pairs. (Please remember I ROUNDED so e..g if you say "I got 1.9M is that OK" then I will worry about you)

In [2]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize
from collections import Counter
from itertools import permutations

# Now, let's tokenize Shakespeare's plays
def distinct_tokens(filename= 'Shakespeare.txt'):
  """
  The function takes in a txt file and returns None. It prints out the total number of
  distinct tokens and total number of distinct token pairs.
  """
  word_count = Counter()
  token_pairs = set() # a set of token pairs
  with open(filename) as f:
    for line in f:
      t = simple_tokenize(line)
      word_count.update(t)
      token_pairs.update(
          set((x,y) for x,y in permutations(t,2) if x != y)
          )
  print("Total distinct tokens: ", len(word_count))
  print("Total distinct token pairs: ", len(token_pairs))

# extend this code to answer Question 1.
# when your code is executed, it should print the number of distinct tokens and the number of distinct token pairs

In [ ]:
distinct_tokens()

Total distinct tokens:  25975
Total distinct token pairs:  1969760


---

#### Question 2 (6/10 marks):
Next, write Python code to answer the one-token and two-token queries described above, for 'Shakespeare.txt'.   The code cell below contains some starter code that implements a simple text-based query interface.  It allows a user to ask a series of one-token or two-token queries.   Try running the starter code to get a sense of how the interface behaves.    

Your task is to write code to read and tokenize 'Shakespeare.txt', record information that will allow both types of PMI queries to be answered, and then answer queries that are posed through the query interface.  Make sure that your code is well commented, so that it will be clear to the markers.

If you cannot get answers to both types of queries working, try to get at least one type working, for partial credit.

Please appropriately handle cases where the token being queried isn't in the file. Print an appropriate error message and continue reading input.
(We won't mark the text of your error messages, but it's bad practice to crash if someone has a typo)

Sanity checks:

```
Input 1 or 2 space-separated tokens (return to quit): the end
n(the,end) = 157
PMI(the,end) = 0.3505058356267105
```

If you got n=147 then you didn't use `simple_tokenize()`, you used `split()` and were thrown off by puntuation and capitlization.

(If you haven't been told this before, floating point numbers are inexact. You do not need to get exactly those digits, but should match at least 4 of them! Make sure you used log base 10)

If your PMI is 2.9ish then you probably are counting how many times each token occurrs rather than how many LINES contain that token.

Because of the phrase "at rope's end", you should also expect that "rope's" doesn't really show up outside this phrase, so it will have a high PMI with "end".

```
n(end,rope's) = 5, PMI(end,rope's) = 2.5ish
```


In [15]:
from collections import defaultdict

a = defaultdict(int)
print(a)
print(a['bleh'])
a['choom'] =1
print(a)
a['choom'] +=1
print(a)

print(dict(a))

defaultdict(<class 'int'>, {})
0
defaultdict(<class 'int'>, {'bleh': 0, 'choom': 1})
defaultdict(<class 'int'>, {'bleh': 0, 'choom': 2})
{'bleh': 0, 'choom': 2}


In [4]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize
# the log function for computing PMI
# for the sake of consistency across solutions, please use log base 10
from math import log

###################################################################################################################
#  replace this with your PMI analysis code, so that you can support the user interface below
#  it should read and tokenize Shakespeare.txt, and store enough information in Python data structures
#  to allow you to answer the PMI queries below
###################################################################################################################
def mapper(x, filename='Shakespeare.txt'):


def findPMI(x="", y="", filename='Shakespeare.txt', threshold):
  """
  The function findPMI takes in two strings, x and y, representing tokens. It
  returns float representing the Pointwise Mutual Information of tokens x and y.

  findPMI: str str -> float
  """
  # find the total number of lines
  nLines = 0
  xNum = 0
  yNum = 0
  xyNum = 0
  with open(filename) as f:
    for line in f:
      nLines += 1
      t = simple_tokenize(line)
      if (x in t) and (y in t):
        xNum += 1
        yNum += 1
        xyNum += 1
      elif (x in t) and (not(y in t)):
        xNum += 1
      else:
        yNum += 1
  p_x = xNum / nLines
  p_y = yNum / nLines
  p_xy = xyNum / nLines

  return math.log(p_xy/(p_x * p_y))

def mapper(line):
  """
  The function mapper takes in a string, line, which represents a line of text
  and emits (word, stripes_dict). Where stripes_dict is a dictionary of all the
  cooccurences of word.
  """
  tokens = simple_tokenize(line)
  unique_tokens = list(set(tokens))
  results = []
  for token in unique_tokens:
    stripe = defaultdict(int)
    for other_token in unique_tokens:
      if token != other_token:
        stripe[other_token] += 1
    results.append((token, dict(stripe)))
  return results

  def reducer(stripesList):
    # we shuffle the
    return None

###################################################################################################################
#  the user interface below defines the types of PMI queries that users can ask
#  you will need to modify it - where indicated - to access the results of your PMI analysis (above)
#  so that the queries can be answered
###################################################################################################################

while True:
    q = input("Input 1 or 2 space-separated tokens (return to quit): ")
    if len(q) == 0:
        break
    q_tokens = simple_tokenize(q)
    if len(q_tokens) == 1:
        threshold = 0
        while threshold <= 0:
            try:
                threshold = int(input("Input a positive integer frequency threshold: "))
            except ValueError:
                print("Threshold must be a positive integer!")
                continue

        # Put code here to answer a One-Token Query with token q_tokens[0] and the specified threshold,
        # and output the result.

        # we initialize a variable n_count to keep track of how many lines
        # q_tokens[0] is found in.
        n_count = 0

        # open the text file and tokenize each line
        with open('Shakespeare.txt') as f:
          for line in f:
            t = simple_tokenize(line)
            # the if statement checks if the word q_tokens[0] is found in line.
            # If found, it updates n_count by 1.
            if q_tokens[0] in t:
              n_count += 1

            # MapReduce
            # Map
            mapped_stripes = mapper(line)

            # Shuffling
            grouped = defaultdict(list)
            for key, stripe in mapped_stripes:
              grouped[key].append(stripe)

            # Reduce



        # pmi_xy = findPMI(x="", y="", filename='Shakespeare.txt', threshold)

        # The print() statements below exist to show you the desired output format.
        # Replace them with your own output code, which should produce results in a similar format.
        print("  n({0}) = {1}".format(q_tokens[0], n_count))
        print("  high PMI tokens with respect to {0} (threshold: {1}):".format(q_tokens[0],threshold))
        print("    n({0},token1) = XXX,  PMI({0},token1) = Y.YYY".format(q_tokens[0]))
        print("    n({0},token2) = XXX,  PMI({0},token2) = Y.YYY".format(q_tokens[0]))
        print("    n({0},token3) = XXX,  PMI({0},token3) = Y.YYY".format(q_tokens[0]))
        print("    n({0},token4) = XXX,  PMI({0},token4) = Y.YYY".format(q_tokens[0]))
        print("    n({0},token5) = XXX,  PMI({0},token5) = Y.YYY".format(q_tokens[0]))
        # in the above, all XXX values should be at least as large as the threshold

    elif len(q_tokens) == 2:
        # Put code here to answer a Two-Token Query with tokens q_tokens[0] and q_tokens[1]
        # As was the case for the One-Token query, the print statements below show the desired output format
        # Replace them with your own output code

        # we initialize a variable n_count2 to keep track of how many lines
        # the token pair (q_tokens[0],q_tokens[1]) is found in
        n_count2 = 0

        # open the text file and tokenize each line
        with open('Shakespeare.txt') as f:
          for line in f:
            t = simple_tokenize(line)
            if (q_tokens[0] in t) and (q_tokens[1] in t):
              n_count2 += 1
          None

        print("  n({0},{1}) = {2}".format(q_tokens[0],q_tokens[1], n_count2))
        print("  PMI({0},{1}) = Y.YYY".format(q_tokens[0],q_tokens[1]))
    else:
        print("Input must consist of 1 or 2 space-separated tokens!")


SyntaxError: parameter without a default follows parameter with a default (ipython-input-231846678.py, line 12)

---

#### Question 3 (2/10 marks):

Suppose that you try to run your PMI analysis on larger files:  say, 10 times, or 100 times, or 1000 times larger than 'Shakespeare.txt'.    As the input file grows larger, what will happen to the execution of your program?   Will it get slower?   How much slower?   Will it eventually fail to run?   If so, why?

Consider two seperate parts: The initial processing of the file, and then the time required to run the queries themselves.
(If you did not do any initial processing and reloaded everything for each query, then please go fix that, that's not okay.)


In the cell below, briefly (one or two paragraphs), discuss what will happen if the input to your analysis grows.  We're not looking for an exact or empirical analysis of the behaviour of your program as a function of input size.  Rather, we are looking for a discussion of trends and limits.

(Tip since we probably haven't got that far in lectures yet: Vocabulary size for a document actually grows unbounded, proportional to the log of the document size!)

#### Answer to Question 3:

As the input size grows, the program will execute slower for both the processing of the file and execution of the program. However, the impact is different for the two cases.

As larger files are inputted, the processing time increases significantly as the different permutataions of words have to be verified for co-occurences at each line and the vocabulary size increases unbounded proportional to the log(document size). Hence, for a significantly larger file, the program will fail as it will run out of memory.

For query execution, this will remain fast as it will rely on returning precomputed results. However, since the vocabulary grows proportionally to the log(document size) there could be memory bloat. The procomputed data becomes too large and there is insufficient memory for query execution.

---
Don't forget to save your workbook!   (It's a good idea to do this regularly, while you are working.)   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.